# Submit Nanoribbon Workchain

In [ ]:
%aiida

In [ ]:
# General imports
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from ase.data import vdw_radii
from IPython.display import display, clear_output, HTML
import itertools

# AiiDA & AiiDA lab imports
from aiida.orm import SinglefileData

from nanoribbon.viewers import Smiles2GNRWidget
from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget, StructureBrowserWidget, StructureUploadWidget, SubmitButtonWidget, SmilesWidget
from aiida.engine import submit

# Work Chains
NanoribbonWorkChain = WorkflowFactory('nanoribbon')

In [ ]:
structure_selector = StructureManagerWidget(importers=[
    ("SMILES2GNR", Smiles2GNRWidget()),
    ("Import from computer", StructureUploadWidget()),
    ("AiiDA database", StructureBrowserWidget()),
    ],
    storable=False,node_class='StructureData')
display(structure_selector)

In [ ]:
style = {'description_width': '120px'}
layout = {'width': '70%'}

pw_code_dropdown = CodeDropdown(input_plugin='quantumespresso.pw')
pp_code_dropdown = CodeDropdown(input_plugin='quantumespresso.pp')
projwfc_code_dropdown = CodeDropdown(input_plugin='quantumespresso.projwfc')

text_calc_description = ipw.Text(description='Calculation Name: ',
                          placeholder='A great name.',
                          style=style, layout=layout)
display(ipw.VBox([pw_code_dropdown,pp_code_dropdown,projwfc_code_dropdown, text_calc_description]))

In [ ]:
def on_submit():
    with submit_out:
        clear_output()
        if not structure_selector.structure_node:
            print("Please select a structure.")
            return
        if not pw_code_dropdown.selected_code:
            print("Please select a pw code.")
            return
        if not pp_code_dropdown.selected_code:
            print("Please select a pp code.")
            return
        if not projwfc_code_dropdown.selected_code:
            print("Please select a projwfc code.")
            return        
        if len(text_calc_description.value) < 3:
            print("Please enter a calculation name.")
            return

        #pw_code, pp_code, projwfc_code = qe_code_groups[drop_codes.value]
        builder = NanoribbonWorkChain.get_builder()
        builder.pw_code = pw_code_dropdown.selected_code
        builder.pp_code = pp_code_dropdown.selected_code
        builder.projwfc_code = projwfc_code_dropdown.selected_code
        builder.structure = structure_selector.structure_node
        builder.precision = Float(1.0) #Float(slider_precision.value)
        builder.pseudo_family = Str('TEST_PSEUDO') # Str('SSSP_precision_v1.0')
        builder.metadata = {
            "description": text_calc_description.value,
            "label": "NanoribbonWorkChain",
        }
        return builder

In [ ]:
btn_submit = SubmitButtonWidget(process=NanoribbonWorkChain,widgets_values=on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)